In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession

scala_version = '2.12'
spark_version = '3.5.0'
packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.6.0'
]
spark = SparkSession.builder.master('local').appName('Bigmart Sales Prediction').config('spark.jars.packages', ','.join(packages)).getOrCreate()
spark

your 131072x1 screen size is bogus. expect trouble
24/01/01 22:16:45 WARN Utils: Your hostname, NamThien resolves to a loopback address: 127.0.1.1; using 172.21.225.129 instead (on interface eth0)
24/01/01 22:16:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/leducmanh/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/leducmanh/.ivy2/cache
The jars for the packages stored in: /home/leducmanh/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a1e592d2-5a31-4c27-8afb-c14ca5c89c80;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.0 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
	found org.apache.kafka#kafka-clients;3.6.0 in central
	found com.github.luben#zstd-jni;1.5.5-1 in 

In [2]:
from kafka import KafkaConsumer

kafka_topic_name = 'turtorial15'
kafka_bootstrap_servers = 'localhost:9092'

consumer = KafkaConsumer(kafka_topic_name, bootstrap_servers=kafka_bootstrap_servers)

In [3]:
from pyspark.ml.regression import LinearRegressionModel

# Load the trained model
model_path = './linear_regression_model'
model = LinearRegressionModel.load(model_path)

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, TimestampType

# Define the schema based on the sample record
schema = StructType([
    StructField('Item_Identifier', StringType(), True),
    StructField('Item_Weight', DoubleType(), True),
    StructField('Item_Fat_Content', StringType(), True),
    StructField('Item_Visibility', DoubleType(), True),
    StructField('Item_Type', StringType(), True),
    StructField('Item_MRP', DoubleType(), True),
    StructField('Outlet_Identifier', StringType(), True),
    StructField('Outlet_Establishment_Year', IntegerType(), True),
    StructField('Outlet_Size', StringType(), True),
    StructField('Outlet_Location_Type', StringType(), True),
    StructField('Outlet_Type', StringType(), True),
    StructField('time_interval', StringType(), True)
])

In [5]:
from pyspark.ml.feature import VectorAssembler, StandardScaler

train_all = spark.read.option('delimiter', ',').option('header', 'true').csv('./Train.csv', schema = schema)
train_all = train_all.na.fill(0)

featureCols = [ 'Item_Weight', 'Item_Visibility', 'Item_MRP']
assembler_all = VectorAssembler(inputCols=featureCols, outputCol='features') 
assembled_df_all = assembler_all.transform(train_all)

In [6]:
from pyspark.sql.functions import isnan, when, count, col
from datetime import datetime
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.sql import Row
import json

# Infinite loop to continuously consume messages
for message in consumer:
    try:
        # Assuming your Kafka message is in the variable 'message'
        message_value = message.value.decode('utf-8')  # Decode bytes to string
        data_dict = json.loads(message_value)  # Convert JSON string to dictionary

        # Convert the 'time_interval' field to a timestamp
        data_dict['time_interval'] = datetime.strptime(data_dict['time_interval'], '%Y-%m-%d %H:%M:%S')

        # Create a Row object from the dictionary
        row = Row(**data_dict)

        # Create a DataFrame from the Row with the specified schema
        df = spark.createDataFrame([row], schema=schema)
        df = df.na.fill(0)

        featureCols = [ 'Item_Weight', 'Item_Visibility', 'Item_MRP']
        # put features into a feature vector column
        assembler = VectorAssembler(inputCols=featureCols, outputCol='features')
        assembled_df = assembler.transform(df)

        # Initialize the `standardScaler`
        standardScaler = StandardScaler(inputCol='features', outputCol='features_scaled')

        # Fit the DataFrame to the scaler
        scaled_df = standardScaler.fit(assembled_df_all).transform(assembled_df)

        # Make predictions using the model
        predictions = model.transform(scaled_df)
        predandlabels = predictions.select('Item_Identifier', 'Item_Outlet_Sales_pred')

        # Print the resulting DataFrame
        predandlabels.show()

    except Exception as e:
        print(f'Error processing message: {str(e)}')

24/01/01 22:17:16 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


+---------------+----------------------+
|Item_Identifier|Item_Outlet_Sales_pred|
+---------------+----------------------+
|          FDO16|     989.5376430330754|
+---------------+----------------------+



+---------------+----------------------+
|Item_Identifier|Item_Outlet_Sales_pred|
+---------------+----------------------+
|          FDU07|    1476.1022366830637|
+---------------+----------------------+



+---------------+----------------------+
|Item_Identifier|Item_Outlet_Sales_pred|
+---------------+----------------------+
|          NCB07|    1677.8731056157994|
+---------------+----------------------+

+---------------+----------------------+
|Item_Identifier|Item_Outlet_Sales_pred|
+---------------+----------------------+
|          FDU48|     1163.685294304299|
+---------------+----------------------+

+---------------+----------------------+
|Item_Identifier|Item_Outlet_Sales_pred|
+---------------+----------------------+
|          NCM43|    1595.0339588677668|
+---------------+----------------------+

+---------------+----------------------+
|Item_Identifier|Item_Outlet_Sales_pred|
+---------------+----------------------+
|          FDR31|    1317.9124892633904|
+---------------+----------------------+

+---------------+----------------------+
|Item_Identifier|Item_Outlet_Sales_pred|
+---------------+----------------------+
|          DRK49|    418.46967764010105|
+-----------